In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
currDir = os.path.dirname(os.path.realpath("__file__"))
rootDir = os.path.abspath(os.path.join(currDir, '..'))
sys.path.insert(1, rootDir)

In [3]:
import numpy as np
import pandas as pd
from src.utils.file_utils import load_df, save_df, load_model

In [4]:
ds1 = load_df("dataset1.csv")
ds2 = load_df("dataset2.csv")

In [5]:
ds1.describe()

,ei,to,td,tf,vers
count,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06
mean,1.854442e+00,2.677224e+02,7.155500e+01,1.216666e+01,4.500000e+00
std,3.186181e+00,1.555065e+03,4.702838e+01,1.125711e+01,2.872282e+00
min,3.479820e-01,1.000000e+00,1.000000e-01,1.000000e+00,0.000000e+00
25%,9.330000e-01,3.000000e+00,3.430000e+01,3.968246e+00,2.000000e+00
50%,1.354955e+00,5.000000e+00,6.850000e+01,9.000000e+00,4.500000e+00
75%,1.354955e+00,5.500000e+01,1.027000e+02,1.700000e+01,7.000000e+00
max,2.317735e+01,1.440000e+04,2.600000e+02,6.000000e+01,9.000000e+00


In [6]:
ds2.describe()

,speed,fe,tt,vers
count,504000.000000,504000.000000,504000.000000,504000.000000
mean,65.260247,0.952462,729.525361,3.000000
std,192.803630,0.308689,2322.622952,2.000002
min,2.764055,0.347982,0.000000,0.000000
25%,7.640000,0.787200,17.167500,1.000000
50%,14.335000,0.887243,58.465000,3.000000
75%,34.130000,1.210000,99.992500,5.000000
max,1164.856900,1.889800,14397.600100,6.000000


In [66]:
def probGroundTruth(ds, feats, featsValQuarts, numVers, catVals):
    
    total = len(ds.index)
    rows = []
    catRows = {}
    namesCol = []
    numCats = len(catVals)
    indexMap = {}
    for i, feat in enumerate(feats):
        if i < numCats:
            vals = catVals[i]
            indexMap[i] = len(vals)
            for val in vals:
                namesCol.append(feat + "=" + val)
                rows.append(np.zeros(numVers))
        else:
            namesCol.append(feat + "-Q1")
            namesCol.append(feat + "-Q2")
            namesCol.append(feat + "-Q3")
            namesCol.append(feat + "-Q4")
            rows.append(np.zeros(numVers))
            rows.append(np.zeros(numVers))
            rows.append(np.zeros(numVers))
            rows.append(np.zeros(numVers))
            indexMap[i] = 4
    #count = 0
    #print(total)
    for index,row in ds.iterrows():
        rowIndex = 0
        for i, feat in enumerate(feats):
            #print("ri:" + str(rowIndex) + " feat:" + feat)
            if i < numCats:
                for j, val in enumerate(featsValQuarts[i]):
                    if row[feat] == val:
                        index = rowIndex + j
                        break
                #print("ind" + str(index))
                rows[index][int(row["vers"])] += 1
                rowIndex += indexMap[i]
            else:
                if row[feat] <= featsValQuarts[i][0]:
                    index = rowIndex
                elif row[feat] <= featsValQuarts[i][1]:
                    index = rowIndex + 1
                elif row[feat] <= featsValQuarts[i][2]:
                    index = rowIndex + 2
                else:
                    index = rowIndex + 3
                #print("ind" + str(index))
                rows[index][int(row["vers"])] += 1
                rowIndex += 4
        #count += 1
        #print(count)
    data = []
    for row in rows:
        dataRow = []
        for val in row:
            dataRow.append(val / total)
        data.append(dataRow)
    data = pd.DataFrame(data, columns=range(numVers), index=namesCol)
    return data
        

In [52]:
feats1 = ["mode", "ei", "to", "td", "tf"]
feats1Quarts = []
feats1Quarts.append(["Private Vehicle", "Taxi/Shuttle/Private Transit", "bus/amtrak bus/transit bus",
                                "rail/trolly/pastco", "school bus", "subway/train/pastco"]) #mode
feats1Quarts.append([0.933,1.354955,26]) #ei (only 3 buckets because the 50% quartile is the same as the 75% quartile)
feats1Quarts.append([3,5,55]) #to
feats1Quarts.append([34.3,68.5,102.7]) #td
feats1Quarts.append([3.968246,9,17,]) #tf

catVals1 = [["Private Vehicle", "Taxi/Shuttle/Private Transit", "bus/amtrak bus/transit bus",
                                "rail/trolly/pastco", "school bus", "subway/train/pastco"]]


feats2 = ["mode", "speed", "fe", "tt"]
feats2Quarts = []
feats2Quarts.append(["Auto", "Bus", "Commuter Rail", "Ferry", "Subway", "Transfer Bus"]) #mode
feats2Quarts.append([7.64,14.335,34.13]) #speed)
feats2Quarts.append([0.7872,0.887243,1.21]) #fe
feats2Quarts.append([17.17675,58.465,99.9925]) #tt

catVals2 = [["Auto", "Bus", "Commuter Rail", "Ferry", "Subway", "Transfer Bus"]]


In [69]:
probs1 = probGroundTruth(ds1,feats1,feats1Quarts,10,catVals1)

In [70]:
print(probs1)

                                          0         1         2         3  \
mode=Private Vehicle               0.050000  0.050000  0.050000  0.050000   
mode=Taxi/Shuttle/Private Transit  0.010000  0.010000  0.010000  0.010000   
mode=bus/amtrak bus/transit bus    0.010000  0.010000  0.010000  0.010000   
mode=rail/trolly/pastco            0.010000  0.010000  0.010000  0.010000   
mode=school bus                    0.010000  0.010000  0.010000  0.010000   
mode=subway/train/pastco           0.010000  0.010000  0.010000  0.010000   
ei-Q1                              0.030000  0.030000  0.020000  0.030000   
ei-Q2                              0.060000  0.060000  0.070000  0.060000   
ei-Q3                              0.010000  0.010000  0.010000  0.010000   
ei-Q4                              0.000000  0.000000  0.000000  0.000000   
to-Q1                              0.040000  0.010000  0.040000  0.040000   
to-Q2                              0.020000  0.000000  0.020000  0.020000   

In [67]:
probs2 = probGroundTruth(ds2,feats2,feats2Quarts,7,catVals2)

In [68]:
print(probs2)

                           0         1         2         3         4  \
mode=Auto           0.023810  0.023810  0.023810  0.023810  0.023810   
mode=Bus            0.023810  0.023810  0.023810  0.023810  0.023810   
mode=Commuter Rail  0.023810  0.023810  0.023810  0.023810  0.023810   
mode=Ferry          0.023810  0.023810  0.023810  0.023810  0.023810   
mode=Subway         0.023810  0.023810  0.023810  0.023810  0.023810   
mode=Transfer Bus   0.023810  0.023810  0.023810  0.023810  0.023810   
speed-Q1            0.047619  0.047619  0.000000  0.047619  0.047619   
speed-Q2            0.023810  0.023810  0.000000  0.023810  0.023810   
speed-Q3            0.071429  0.071429  0.000000  0.071429  0.071429   
speed-Q4            0.000000  0.000000  0.142857  0.000000  0.000000   
fe-Q1               0.071429  0.071429  0.071429  0.071429  0.071429   
fe-Q2               0.000000  0.000000  0.000000  0.023810  0.000000   
fe-Q3               0.047619  0.047619  0.047619  0.000000  0.04

In [72]:
save_df(probs1, "ground-truth-lime-1.csv", withIndex=True)
save_df(probs2, "ground-truth-lime-2.csv", withIndex=True)

df successfully saved | filename: ground-truth-lime-1.csv, dir: C:\Users\archg\school\senoir\xai-senior-design\data
df successfully saved | filename: ground-truth-lime-2.csv, dir: C:\Users\archg\school\senoir\xai-senior-design\data
